# Dependencies

In [1]:
using Base
using StaticArrays

# MultiDual

In [2]:
mutable struct MultiDual{N,T}
	val::T
	# SVector is static vector which lives on the stack
	grads::SVector{N,T} 
end

In [18]:
function Base.:+(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f.val+g.val, f.grads+g.grads)
end

function Base.:*(f::MultiDual{N,T}, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f.val*g.val, f.val*g.grads+f.grads*g.val)
end

function Base.:*(f::Number, g::MultiDual{N,T}) where {N,T}
    return MultiDual{N,T}(f*g.val, f*g.grads)
end

function Base.:*(f::MultiDual{N,T}, g::Number) where {N,T}
    return MultiDual{N,T}(f.val*g, f.grads*g)
end

function Base.:^(f::MultiDual{N,T}, g::Real) where {N,T}
    return Base.power_by_squaring(f,g)
end

In [22]:
stencil(a,b,c) = 2*a * 3*b * 4*c

arr_in = rand(9,1)
vals_out = zeros(7,1)
grads_out = zeros(7,3)

for i=2:length(arr_in)-1
    output = stencil(MultiDual(arr_in[i-1], SVector(1.,0.,0.)), MultiDual(arr_in[i], SVector(0.,1.,0.)), MultiDual(arr_in[i+1], SVector(0.,0.,1.)))
    vals_out[i-1] = output.val
    grads_out[i-1,:] = output.grads
end

In [23]:
vals_out


7×1 Matrix{Float64}:
  7.997894530850245
 12.116094902743747
  0.138586989684442
  0.11212640254683372
  0.08120614816299015
  2.807202617019824
  0.600623095137713

In [24]:
grads_out

7×3 Matrix{Float64}:
 14.1503    11.0933     9.78001
 16.8053    14.8158    14.1503
  0.169467   0.161854  16.8053
  0.130951  13.5966     0.169467
  9.84719    0.122735   0.130951
  4.24279    4.52683    9.84719
  0.968552   2.10688    4.24279